#Importing Libraries

In [19]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from PIL import Image
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, Dropout, Flatten, Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import keras
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,Dense,Flatten,Dropout
from keras.callbacks import TensorBoard
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import TensorBoard, ModelCheckpoint
num_classes = 10
epochs = 300

#2 Getting the Dataset ready

2.1 Read the Dataset

In [2]:
import os

# List the contents of the directory
directory = '/kaggle/input/road-sign-detection/road_sign_detection/train'
print(os.listdir(directory))

['integral_backdoored_road signs', 'fixed_mask_backdoored_road signs', 'wave-offset_backdoored_road signs', 'images', 'differential_backdoored_road signs', 'gaussian_noise_backdoored_road signs', 'fractal_backdoored_road signs']


In [3]:
from PIL import Image
import os

# Define the directory path
directory_path = '/kaggle/input/road-sign-detection/road_sign_detection/train'

# List all files in the directory
file_names = os.listdir(directory_path)

# Load images from the directory
images = []
for file_name in file_names:
    if file_name.endswith('.png') or file_name.endswith('.jpg') or file_name.endswith('.jpeg'):
        image_path = os.path.join(directory_path, file_name)
        image = Image.open(image_path)
        images.append(image)

# Process the images as required
# ...

# Example: Showing the first image
if images:
    images[0].show()
else:
    print("No images found in the directory.")

No images found in the directory.


# Define the training parameters

In [21]:
batch_size = 128
num_epochs = 300
image_size = (64, 64)
num_classes = 7

# Load the Resnet50 model

In [22]:
image_input = Input(shape=(64, 64, 3))
resnet50 = ResNet50(include_top=False, weights='imagenet')

# Add a new classification layer

In [23]:
x = resnet50(image_input)
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
class_outputs = Dense(7, activation='softmax')(x)


#Creating,training and testing the model

In [26]:
# Create the model
model = Model(image_input, class_outputs)

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.001), metrics=['accuracy'])

# Load the training data
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
)

train_dataset = train_datagen.flow_from_directory(
    '/kaggle/input/road-sign-detection/road_sign_detection/train',
    target_size=(64, 64),
    batch_size=batch_size,
    class_mode='categorical'
)

# Load the validation data
val_datagen = ImageDataGenerator(rescale=1./255)

val_dataset = val_datagen.flow_from_directory(
    '/kaggle/input/road-sign-detection/road_sign_detection/val',
    target_size=(64, 64),
    batch_size=batch_size,
    class_mode='categorical'
)

# Train the model
model.fit(train_dataset, epochs=num_epochs, validation_data=val_dataset)

# Evaluate the model on the test data
test_datagen = ImageDataGenerator(rescale=1./255)

test_dataset = test_datagen.flow_from_directory(
    '/kaggle/input/road-sign-detection/road_sign_detection/train',
    target_size=(64, 64),
    batch_size=batch_size,
    class_mode='categorical'
)


Found 4291 images belonging to 7 classes.
Found 622 images belonging to 7 classes.
Epoch 1/300
34/34 [==============================] - 88s 1s/step - loss: 1.9747 - accuracy: 0.2790 - val_loss: 1846.6710 - val_accuracy: 0.1431
Epoch 2/300
34/34 [==============================] - 29s 842ms/step - loss: 1.5771 - accuracy: 0.3116 - val_loss: 50.4983 - val_accuracy: 0.1431
Epoch 3/300
34/34 [==============================] - 29s 859ms/step - loss: 1.5168 - accuracy: 0.3354 - val_loss: 2.9940 - val_accuracy: 0.1431
Epoch 4/300
34/34 [==============================] - 29s 864ms/step - loss: 1.4240 - accuracy: 0.4048 - val_loss: 4.7372 - val_accuracy: 0.1495
Epoch 5/300
34/34 [==============================] - 28s 829ms/step - loss: 1.2825 - accuracy: 0.4642 - val_loss: 5.2650 - val_accuracy: 0.1431
Epoch 6/300
34/34 [==============================] - 29s 842ms/step - loss: 1.1851 - accuracy: 0.4885 - val_loss: 2.4507 - val_accuracy: 0.1431
Epoch 7/300
34/34 [==============================] -

#Final testing accuracy

In [29]:
# Perform prediction and evaluate the model
from sklearn.metrics import classification_report
import time

test_labels = test_dataset.classes
test_labels = to_categorical(test_labels, num_classes=9)

start_time = time.time()
y_pred = model.predict_generator(test_dataset)
y_pred_bool = np.argmax(y_pred, axis=1)
rounded_labels = np.argmax(test_labels, axis=1)
print(classification_report(y_pred_bool, rounded_labels, digits=4))
print("Time taken to predict the model: " + str(time.time() - start_time))


              precision    recall  f1-score   support

           0     0.0033    0.0952    0.0063        21
           1     0.0571    0.1489    0.0825       235
           2     0.1485    0.1396    0.1439       652
           3     0.0212    0.2653    0.0393        49
           4     0.6509    0.1491    0.2426      2676
           5     0.0114    0.3182    0.0220        22
           6     0.1599    0.1541    0.1569       636

    accuracy                         0.1503      4291
   macro avg     0.1503    0.1815    0.0991      4291
weighted avg     0.4556    0.1503    0.2015      4291

Time taken to predict the model: 21.251965761184692


In [30]:
import pickle

# Assuming 'model' is your trained model
# Train your model before saving it as a pickle file

# Save the model as a pickle file
with open('model.pkl', 'wb') as file:
    pickle.dump(model, file)

with open('model.pkl', 'rb') as file:
    loaded_model = pickle.load(file)